In [2]:
import nltk
import random
import feedparser

In [3]:
urls = {
    'mlb': 'https://sports.yahoo.com/mlb/rss.xml',
    'nfl': 'https://sports.yahoo.com/nfl/rss.xml',
}

In [4]:
feedmap = {}
stopwords = nltk.corpus.stopwords.words('english')

def featureExtractor(words):
    features = {}
    for word in words:
        if word not in stopwords:
            features["word({})".format(word)] = True
    return features

In [11]:
sentences = []

for category in urls.keys():
    feedmap[category] = feedparser.parse(urls[category])
    print("downloading {}".format(urls[category]))
    for entry in feedmap[category]['entries']:
        data = entry['summary']
        words = data.split()
        sentences.append((category, words))

downloading https://sports.yahoo.com/mlb/rss.xml
downloading https://sports.yahoo.com/nfl/rss.xml


In [13]:
featuresets = [(featureExtractor(words), category) for category, words in sentences]
random.shuffle(featuresets)

total = len(featuresets)
off = int(total/2)
trainset = featuresets[off:]
testset = featuresets[:off]

In [14]:
classifier = nltk.NaiveBayesClassifier.train(trainset)

print(nltk.classify.accuracy(classifier, testset))

0.8085106382978723


In [19]:
classifier.show_most_informative_features(5)
for (i, entry) in enumerate(feedmap['nfl']['entries']):
    if i < 4:
        features = featureExtractor(entry['title'].split())
        category = classifier.classify(features)
        print('{} -> {}'.format(category, entry['summary']))

Most Informative Features
          word(Saturday) = True              mlb : nfl    =      7.3 : 1.0
              word(back) = True              nfl : mlb    =      6.1 : 1.0
               word(get) = True              mlb : nfl    =      3.1 : 1.0
              word(past) = True              mlb : nfl    =      2.4 : 1.0
             word(first) = True              mlb : nfl    =      2.4 : 1.0
nfl -> Ed Reed's legendary Hall of Fame bust, Champ Bailey's plea for racial understanding and Tony Gonzalez's moving message stood out at Saturday's ceremony.
nfl -> All four entered the Pro Football Hall of Fame on Saturday night.  ''Fitting to be in here with Mr. Johnny Robinson and Champ and Ty,'' Reed said.  A six-time All-Pro, Gonzalez helped revolutionize the position, lining up in traditional tight end spots as well as flanked out or in the
nfl -> Tony Gonzalez probably could have been a star in the NBA.  Instead, he chose the NFL path, and the most productive tight end in history has